<a href="https://colab.research.google.com/github/sindhu819/Spark-2020/blob/main/012467832_RDD_PAIR_of_HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **HW 9:RDD pair Operation** #

To run spark in Colab, we need to first install pyspark


In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [74.3 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 k

In [ ]:
from pyspark import SparkContext
sc = spark.sparkContext

###We will import an csv file to illustrate some more RDD opearations.

In [ ]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'1ilCXuYltyjv1auHn8FY_Csksep9gz03c'}) # replace the id with id of file you want to access
downloaded.GetContentFile('video-years.csv')

## Importing data ##

In [ ]:
records = sc.textFile('video-years.csv')
records.collect() # See what got imported into this RDD

['b6ea95c5-9aea-11e5-a6ca-8b496c707234,"Monsters Inc.",2001',
 'b6d734d1-9aea-11e5-a6ca-8b496c707234,"Stuart Little 2",2002',
 'b6c5cfb5-9aea-11e5-a6ca-8b496c707234,"Piglet\'s Big Movie",2003',
 'b6d22bc1-9aea-11e5-a6ca-8b496c707234,"Jimmy Neutron: Boy Genius",2001',
 'b6c31093-9aea-11e5-a6ca-8b496c707234,"The Country Bears",2002',
 'b6d89469-9aea-11e5-a6ca-8b496c707234,"The Jungle Book 2",2003',
 'b6cd70d5-9aea-11e5-a6ca-8b496c707234,"The Tigger Movie",2000',
 'b6c31095-9aea-11e5-a6ca-8b496c707234,"Tarzan & Jane",2002',
 'b6d9cce1-9aea-11e5-a6ca-8b496c707234,"Finding Nemo",2003']

In [ ]:
records.count()

9

### Create key-value RDD pairs.###

In [ ]:
pair = records.map(lambda x: (x.split(",")[1].replace("\"",""),x.split(",")[2]))
pair.take(9)

[('Monsters Inc.', '2001'),
 ('Stuart Little 2', '2002'),
 ("Piglet's Big Movie", '2003'),
 ('Jimmy Neutron: Boy Genius', '2001'),
 ('The Country Bears', '2002'),
 ('The Jungle Book 2', '2003'),
 ('The Tigger Movie', '2000'),
 ('Tarzan & Jane', '2002'),
 ('Finding Nemo', '2003')]

In [ ]:
type(pair) #IT'S AN RDD


pyspark.rdd.PipelinedRDD

In [ ]:
pair.take(1) #the output of take gives the list
type(pair.take(2))

list

#### Filtering on the second element, i.e., value, and only keep those movies that are released after year 2002 (20%) ####

In [ ]:
rdd=pair.filter(lambda x: x[1]>'2002')
print("The movies that are released after 2002 : %s" %rdd.collect())

The movies that are released after 2002 : [("Piglet's Big Movie", '2003'), ('The Jungle Book 2', '2003'), ('Finding Nemo', '2003')]


#### Use ReduceByKey and mapValues to compute the average value for each key in the RDD pairs (30%)####

In [ ]:
rdd = sc.parallelize([("panda", 0), ("pink" , 3), ("pirate", 3), ("panda", 1), ("pink", 4)])
rdd.collect()

[('panda', 0), ('pink', 3), ('pirate', 3), ('panda', 1), ('pink', 4)]

In [ ]:
rdd_key_count=rdd.mapValues(lambda x:(x,1)) # for rdd pair just normal map works differently.
print(rdd_key_count.collect())
rdd_key_sum=rdd_key_count.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))
print(rdd_key_sum.collect())
rdd_key_count=rdd_key_sum.mapValues(lambda x:x[0]/x[1] )
print(rdd_key_count.collect())


[('panda', (0, 1)), ('pink', (3, 1)), ('pirate', (3, 1)), ('panda', (1, 1)), ('pink', (4, 1))]
[('panda', (1, 2)), ('pink', (7, 2)), ('pirate', (3, 1))]
[('panda', 0.5), ('pink', 3.5), ('pirate', 3.0)]


In [ ]:
rdd_average=rdd.mapValues(lambda x:(x,1)).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).mapValues(lambda x: x[0]/x[1])
print("The average value of each key in the rdd pairs is :%s" %rdd_average.collect())

The average value of each key in the rdd pairs is :[('panda', 0.5), ('pink', 3.5), ('pirate', 3.0)]


##Word count problem ##

####Consider the movie title, duration, rating info below. ####

|Title                    |Duration (mins)|  User Rating|
|----------------         |-----      |-----|
|The Martian              |141            |7.6|
|Bridge of Spies          |141           |8.0 |
|The Imitation Game      |	113          |	8.0|          
|The Wolf of Wall Street  | 180         |	7.9|
|Creed  |132             |8.6|
|John Wick           |101     |7.2|
|The Hundred-Foot Journey| 122 |	7.3|


####Create an RDD out of this table####

In [ ]:
movies = sc.parallelize(['"The Martian", 141, 7.6',
                    '"Bridge of Spies", 141, 8.0',
                    '"The Imitation Game", 113, 8.0',
                    '"The Wolf of Wall Street", 180, 7.9',
                    '"Creed", 132, 8.6',
                    '"John Wick", 101, 7.2',
                    '"The Hundred-Foot Journey", 122, 7.3'])
movies.collect()

['"The Martian", 141, 7.6',
 '"Bridge of Spies", 141, 8.0',
 '"The Imitation Game", 113, 8.0',
 '"The Wolf of Wall Street", 180, 7.9',
 '"Creed", 132, 8.6',
 '"John Wick", 101, 7.2',
 '"The Hundred-Foot Journey", 122, 7.3']

####How many movies are in this list? ####

In [ ]:
movies.count()
# len(movies.collect())

7

####what is the word count of all the movie titles? Use reduceByKey for this part (30%)  #### 
#####Print them out by 'word : total occurence of this word' line by line (all "s need to be removed in each word)#####

In [ ]:
rdd_movies=movies.map(lambda x: x.split(",")[0].replace("\"",""))
rdd_words=rdd_movies.flatMap(lambda x : x.split(" ")).map(lambda x : (x,1)).reduceByKey(lambda x, y : x+y)
#rdd_words.collect()
for i in range (0, rdd_words.count()):
  print ("%s : %d" % (rdd_words.collect()[i][0],rdd_words.collect()[i][1]))

The : 4
Bridge : 1
of : 2
Spies : 1
Journey : 1
Martian : 1
Imitation : 1
Game : 1
Wolf : 1
Wall : 1
Street : 1
Creed : 1
John : 1
Wick : 1
Hundred-Foot : 1


### word count: use combineByKey only for this part(20%) ####

In [ ]:
rdd_movies1=movies.map(lambda x: x.split(",")[0].replace("\"","")).flatMap(lambda x:x.split(" ")).map(lambda x:(x,1))
rdd_words1=rdd_movies1.combineByKey((lambda x: (x,1)),(lambda x, y: (x[0] + y, x[1] + 1)),(lambda x, y: (x[0] + y[0], x[1] + y[1])))
for i in range (0, rdd_words1.count()):
   print ("%s : %d" % (rdd_words1.collect()[i][0],rdd_words1.collect()[i][1][0]))

The : 4
Bridge : 1
of : 2
Spies : 1
Journey : 1
Martian : 1
Imitation : 1
Game : 1
Wolf : 1
Wall : 1
Street : 1
Creed : 1
John : 1
Wick : 1
Hundred-Foot : 1
